In [1]:
# 기본 module 설치
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00


## 마스크드 언어 모델과 토크나이저

In [2]:
# 주의 model과 tokenizer은 항상 mapping 관계
# model에 해당하는 tokenizer를 사용해야한다.
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer
from transformers import FillMaskPipeline

In [3]:
# TFBertForMaskedLM.from_pretrained: [MASK]라고 되어있는 단어를 맞추기 위한 BERT 구조를 Loading
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


### Input of BERT

In [4]:
# 정수 인코딩 수행
# sample text: Soccer is a really fun [MASK]
inputs = tokenizer('Soccer is a really fun [MASK].', return_tensors='tf')

In [5]:
print(inputs['input_ids'])

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


In [6]:
# 세그먼트 encoding 결과 확인

'''
현재 문장이 1개이기 때문에 문장의 길이만큼의 0으로 이루어진 sequence 반환
만약 문장이 2개가 주어진다면 두 번째 문장부터는 1로 시작하는 sequence가 등장했을 것
'''

print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


In [7]:
# Attention mask 확인
# 패딩이 없기 때문에 모두 1
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1]], shape=(1, 9), dtype=int32)


### [MASK] 토큰 예측하기

FillMaskPipeline은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리해서 보여준다

In [9]:
# 파이프라인에 모델과 tokenizer 저장
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

[Mask]위치에 들어갈 상위 5개의 단위 출력하기

In [10]:
pip('Soccer is a really fun [MASK].')

[{'score': 0.7621124386787415,
  'token': 4368,
  'token_str': 'sport',
  'sequence': 'soccer is a really fun sport.'},
 {'score': 0.2034195065498352,
  'token': 2208,
  'token_str': 'game',
  'sequence': 'soccer is a really fun game.'},
 {'score': 0.01220853440463543,
  'token': 2518,
  'token_str': 'thing',
  'sequence': 'soccer is a really fun thing.'},
 {'score': 0.001863025827333331,
  'token': 4023,
  'token_str': 'activity',
  'sequence': 'soccer is a really fun activity.'},
 {'score': 0.0013354859547689557,
  'token': 2492,
  'token_str': 'field',
  'sequence': 'soccer is a really fun field.'}]

In [11]:
pip('The Avengers is a really fun [MASK].')

[{'score': 0.256289541721344,
  'token': 2265,
  'token_str': 'show',
  'sequence': 'the avengers is a really fun show.'},
 {'score': 0.17284126579761505,
  'token': 3185,
  'token_str': 'movie',
  'sequence': 'the avengers is a really fun movie.'},
 {'score': 0.11107687652111053,
  'token': 2466,
  'token_str': 'story',
  'sequence': 'the avengers is a really fun story.'},
 {'score': 0.07248992472887039,
  'token': 2186,
  'token_str': 'series',
  'sequence': 'the avengers is a really fun series.'},
 {'score': 0.07046639174222946,
  'token': 2143,
  'token_str': 'film',
  'sequence': 'the avengers is a really fun film.'}]

In [12]:
pip('I went to [MASK] this morning.')

[{'score': 0.35730740427970886,
  'token': 2147,
  'token_str': 'work',
  'sequence': 'i went to work this morning.'},
 {'score': 0.23304425179958344,
  'token': 2793,
  'token_str': 'bed',
  'sequence': 'i went to bed this morning.'},
 {'score': 0.12845070660114288,
  'token': 2082,
  'token_str': 'school',
  'sequence': 'i went to school this morning.'},
 {'score': 0.06230573728680611,
  'token': 3637,
  'token_str': 'sleep',
  'sequence': 'i went to sleep this morning.'},
 {'score': 0.04695259407162666,
  'token': 2465,
  'token_str': 'class',
  'sequence': 'i went to class this morning.'}]

## 한국어 BERT의 마스크드 언어 모델(Masked Language Model) 실습

In [13]:
# 새로운 모델 loading
# 한국어로 학습된 BERT
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [14]:
# input
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')

In [16]:
# 정수 인코딩 결과 확인
print(inputs['input_ids'])

# 세그먼트 인코딩 결과 확인
print(inputs['token_type_ids'])

# Attention Mask 확인
print(inputs['attention_mask'])

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)
tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)
tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


In [17]:
# [Mask] token 예측하기
# pipline 생성
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [18]:
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.8963510394096375,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.025957586243748665,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010033945553004742,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924389094114304,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844221778213978,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [19]:
pip('나는 오늘 아침에 [MASK]에 출근을 했다.')

[{'score': 0.08012550324201584,
  'token': 3769,
  'token_str': '회사',
  'sequence': '나는 오늘 아침에 회사 에 출근을 했다.'},
 {'score': 0.061240557581186295,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '나는 오늘 아침에 에 출근을 했다.'},
 {'score': 0.01748667098581791,
  'token': 4345,
  'token_str': '공장',
  'sequence': '나는 오늘 아침에 공장 에 출근을 했다.'},
 {'score': 0.016131775453686714,
  'token': 5841,
  'token_str': '사무실',
  'sequence': '나는 오늘 아침에 사무실 에 출근을 했다.'},
 {'score': 0.015360771678388119,
  'token': 3671,
  'token_str': '서울',
  'sequence': '나는 오늘 아침에 서울 에 출근을 했다.'}]

## 구글 BERT의 다음 문장 예측(Next Sentence Prediction)

In [20]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [21]:
# TFBertForNextSentencePrediction.from_pretrained: 두 개의 문장이 이어지는 문장인지 판단하는 BERT 구조 로드
model = TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForNextSentencePrediction.

All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


### Input of BERT

In [22]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "pizza is eaten with the use of a knife and fork. In casual settings, however, it is cut into wedges to be eaten while held in the hand."

In [25]:
# 정수 인코딩 수행하기
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

# 결과 확인하기
print(encoding['input_ids'])

# 다른 embedding 결과 확인하기 - 세그먼트(token_type_ids), Attention mask
print(encoding)

tf.Tensor(
[[  101  1999  3304  1010 10733  2366  1999  5337 10906  1010  2107  2004
   2012  1037  4825  1010  2003  3591  4895 14540  6610  2094  1012   102
  10733  2003  8828  2007  1996  2224  1997  1037  5442  1998  9292  1012
   1999 10017 10906  1010  2174  1010  2009  2003  3013  2046 17632  2015
   2000  2022  8828  2096  2218  1999  1996  2192  1012   102]], shape=(1, 58), dtype=int32)
{'input_ids': <tf.Tensor: shape=(1, 58), dtype=int32, numpy=
array([[  101,  1999,  3304,  1010, 10733,  2366,  1999,  5337, 10906,
         1010,  2107,  2004,  2012,  1037,  4825,  1010,  2003,  3591,
         4895, 14540,  6610,  2094,  1012,   102, 10733,  2003,  8828,
         2007,  1996,  2224,  1997,  1037,  5442,  1998,  9292,  1012,
         1999, 10017, 10906,  1010,  2174,  1010,  2009,  2003,  3013,
         2046, 17632,  2015,  2000,  2022,  8828,  2096,  2218,  1999,
         1996,  2192,  1012,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 58), dtype=int32, nu

In [26]:
# 특수 token 확인해보기
print(tokenizer.cls_token, ':', tokenizer.cls_token_id)
print(tokenizer.sep_token, ':' , tokenizer.sep_token_id)

[CLS] : 101
[SEP] : 102


In [27]:
# 다시 decoding하여 입력 구성 확인
print(tokenizer.decode(encoding['input_ids'][0]))

[CLS] in italy, pizza served in formal settings, such as at a restaurant, is presented unsliced. [SEP] pizza is eaten with the use of a knife and fork. in casual settings, however, it is cut into wedges to be eaten while held in the hand. [SEP]


### Prediction

In [28]:
logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[9.9999714e-01 2.8381855e-06]], shape=(1, 2), dtype=float32)


In [29]:
# 확률이 높은 index 출력하게 하기
# 이어진다: 0
# 이어지지 않는다: 1
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


In [31]:
# 상관없는 두 개의 문장 예측해보기
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

tf.Tensor([[1.2606483e-04 9.9987388e-01]], shape=(1, 2), dtype=float32)
최종 예측 레이블 : [1]
